
- 高斯分布型：用于classification问题，假定属性/特征是服从正态分布的。
- 多项式型：用于离散值模型里。比如文本分类问题里面我们提到过，我们不光看词语是否在文本中出现，也得看出现的次数。如果总词数为n，出现词数为m的话，说起来有点像掷骰子n次出现m次这个词的场景。
- 伯努利型：这种情况下，就如之前博文里提到的bag of words处理方式一样，最后得到的特征只有0(没出现)和1(出现过)。


In [13]:
# coding=utf-8
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
# iris dataset
iris = datasets.load_iris()
iris.data[:5]

# supposing that sepal length, sepal width, petal length, petal width are complied with Gaussian Distribution
# 4个量独立且服从高斯分布，用贝叶斯分类器建模
# Gaussian Model
gnb = GaussianNB()

# train
clf = gnb.fit(iris.data, iris.target)

# predict
y_pred = clf.predict(iris.data)

right_num = (iris.target == y_pred).sum()

#print "Total num of test data : {:d} , naive bayes accuracy : {:.2f}%".format(iris.data.shape[0], float(right_num)/iris.data.shape[0])

#assert 'Total num of test data :150 , naive bayes accuracy :0.96%'

In [14]:
from sklearn.naive_bayes import GaussianNB

#0:晴 1:阴 2:降水 3:多云
data_table = [["date", "weather"],
              [1, 0],
              [2, 1],
              [3, 2],
              [4, 1],
              [5, 2],
              [6, 0],
              [7, 0],
              [8, 3],
              [9, 1],
              [10, 1]]
#当天的天气
X = [[0], [1], [2], [1], [2], [0], [0], [3], [1]]

#当天的天气对应后一天的天气
Y = [1, 2, 1, 2, 0, 0, 3, 1, 1]

# X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
# Y = np.array([1, 1, 1, 2, 2, 2])
#现在我们把训练数据，和对应的分类放入分类器中进行训练
clf = GaussianNB().fit(X, Y)

#预测当天是晴天后一天的天气
p = [[1]]
print clf.predict(p)

[2]


### Mapping relationships

* The following process compute data of mapping relationships provided by Sogou
[DataSet](http://www.sogou.com/labs/dl/cs.html)(in Chinese)

In [31]:
# coding=utf-8
import codecs
def url_label_lst():
    fhand = codecs.open('url_label.txt', encoding='utf-8').read()
    url_label_lst = list()
    fhand = fhand.split('\n')
    for i in range(len(fhand)):
        ul = fhand[i].split('\t')
        url_label_lst.append(ul)
    return url_label_lst

def url_map_label(url_label_lst):
    dct = {}
    labels = []
    for i in range(len(lst) - 1):
        (url,label) = lst[i]
        dct[label] = dct.get(label,1)
    for key in dct.keys():
        labels.append(key)
    labels.sort()
    
    return labels

def labels_num_dict(labels_lst):
    labels_dict = {}
    for i in range(len(labels_lst)):
        labels_dict[labels_lst[i]] = i   
    return labels_dict

lst = url_label_lst()
labels = url_map_label(lst)
    
print 'Total {} labels in the dataset.'.format(len(labels))
print '总共 {} 个新闻类别/标签'.format(len(labels))
print 'Labels are: \n{0}.'.format(labels)

print [['url','label']]
labels_num = labels_num_dict(labels)
print 'labels in number: {}'.format(labels_num)
#sorted([(v,k) for k,v in labels_num.items()])
print labels_num[lst[3][1]]

Total 18 labels in the dataset.
总共 18 个新闻类别/标签
Labels are: 
[u'IT', u'\u4f53\u80b2', u'\u5065\u5eb7', u'\u519b\u4e8b', u'\u56fd\u5185', u'\u56fd\u9645', u'\u5965\u8fd0', u'\u5973\u6027', u'\u5a31\u4e50', u'\u623f\u4ea7', u'\u62db\u8058', u'\u6559\u80b2', u'\u6587\u5316', u'\u65c5\u6e38', u'\u6821\u56ed', u'\u6c7d\u8f66', u'\u793e\u4f1a', u'\u8d22\u7ecf'].
[['url', 'label']]
labels in number: {u'\u56fd\u5185': 4, u'\u6587\u5316': 12, u'\u8d22\u7ecf': 17, u'\u793e\u4f1a': 16, u'\u5a31\u4e50': 8, u'\u4f53\u80b2': 1, u'\u62db\u8058': 10, u'\u6821\u56ed': 14, u'IT': 0, u'\u56fd\u9645': 5, u'\u623f\u4ea7': 9, u'\u6c7d\u8f66': 15, u'\u65c5\u6e38': 13, u'\u519b\u4e8b': 3, u'\u5065\u5eb7': 2, u'\u6559\u80b2': 11, u'\u5973\u6027': 7, u'\u5965\u8fd0': 6}
2


In [48]:
import codecs
import jieba


def replace_label_seg(jsonData, url_label_lst, labels_num):
    for i in range(len(jsonData) - 1):
        # replace url label
        for j in range(len(url_label_lst) - 1):
            if url_label_lst[j][0] in str(jsonData[i]['url']):
                jsonData[i]['url'] = labels_num[url_label_lst[j][1]]
                
        # word segement Default Mode 精确模式
        # some content are empty and it is a list when transforming into json from xml
        if isinstance(jsonData[i]['content'],list) or isinstance(jsonData[i]['contenttitle'],list): 
            jsonData[i]['content'] = ''
            jsonData[i]['contenttitle'] = ''
            continue
        
        seg_con = jieba.cut(jsonData[i]['content'], cut_all=False)
        # TODO 还需要去除停用词
        jsonData[i]['content'] = ' '.join(seg_con)
        
        seg_title = jieba.cut(jsonData[i]['contenttitle'], cut_all=False)
        jsonData[i]['contenttitle'] = ' '.join(seg_title)
    
    return jsonData


import json
# load json data to be cleaned and preprocess
data = open("data.json").read()
jsonData = json.loads(data)
# load labels and transform into number
lst = url_label_lst()
labels = url_map_label(lst)
labels_num = labels_num_dict(labels)

# clean and preprocess
jsonData = replace_label_seg(jsonData, lst, labels_num)

#print jsonData[8]['contenttitle']
# save cleaned data
#cleaned_data = json.dumps(jsonData, indent = 4)

#data_pro = codecs.open('data_processed.json','w', encoding='utf-8')
#data_pro.write(cleaned_data)
#data_pro.close()

print 'Processing completed. Text num is {0}'.format(len(jsonData))

Processing completed. Text num is 963


### The origin data is in xml and GBK format. It is processed into json and utf-8 file via Java

### The above process completed following tasks:
* The url indicates the label of a text. Mapping relation is provided by Sogou. I replace the url with Chinese Label. See method<code>url_label_lst()</code>
* Word segmentation also completed while replacing urls to labels. Jieba is applied to segment
* Finally, save the processed data in json and utf-8 format.

### Next Step is to futher process and train Naive Bayes Classifier

In [50]:
print jsonData[3]['url']

9


In [53]:
#json_proc = codecs.open('data_processed.json','r', encoding='utf-8').read()
#fileData = json.loads(json_proc)

fileWrite = codecs.open('data_processed3.json','w','utf-8')
for i in range(len(jsonData) - 1):
    if len(str(jsonData[i]['url'])) > 0 and len(jsonData[i]['content']) > 0 and len(jsonData[i]['contenttitle']) > 0:
        fileWrite.write(str(jsonData[i]['url']).strip() + '\t' + jsonData[i]['content'].strip() + '\n')

fileWrite.close()    

In [307]:
data2 = open("data_processed2.json").read()
data2 = data2.split('\n')
print len(data2)
for line in range(len(trainText) - 2):
        lable,text = data2[line].split('\t',1)
print lable[0]

961


ValueError: need more than 1 value to unpack

In [260]:
print labels
lables = labels

[u'IT', u'\u4f53\u80b2', u'\u5065\u5eb7', u'\u519b\u4e8b', u'\u56fd\u5185', u'\u56fd\u9645', u'\u5965\u8fd0', u'\u5973\u6027', u'\u5a31\u4e50', u'\u623f\u4ea7', u'\u62db\u8058', u'\u6559\u80b2', u'\u6587\u5316', u'\u65c5\u6e38', u'\u6821\u56ed', u'\u6c7d\u8f66', u'\u793e\u4f1a', u'\u8d22\u7ecf']


In [252]:
#总共有9种新闻类别，我们给每个类别一个编号
def lable2id(lable):
    for i in xrange(len(lables)):
        if lable == lables[i]:
            return i
    raise Exception('Error lable {0}'.format(lable))


def doc_dict():
    return [0] * len(lables)

def mutual_info(N,Nij,Ni_,N_j):
    '''
        计算互信息，这里log的底取为2
    '''
    return Nij * 1.0 / N * math.log(N * (Nij+1)*1.0/(Ni_*N_j))/ math.log(2)

In [265]:
def count_for_cates(trainText, featureFile):
    '''
        遍历文件，统计每个词在每个类别出现的次数，和每类的文档数
        并写入结果特征文件
    '''
    docCount = [0] * len(lables)
    wordCount = collections.defaultdict(lambda:doc_dict())
    #扫描文件和计数
    trainText = trainText.split('\n')
    for line in range(len(trainText) - 2):
        lable,text = trainText[line].strip().split('\t',1)
        index = lable2id(lable[0])        
        words = text.split(' ')
        for word in words:
            wordCount[word][index] += 1
            docCount[index] += 1
    #计算互信息值
    print "计算互信息，提取关键/特征词中，请稍后..."
    miDict = collections.defaultdict(doc_dict())
    N = sum(docCount)
    for k,vs in wordCount.items():
        for i in xrange(len(vs)):
            N11 = vs[i]
            N10 = sum(vs) - N11
            N01 = docCount[i] - N11
            N00 = N - N11 - N10 - N01
            mi = mutual_info(N,N11,N10+N11,N01+N11) + mutual_info(N,N10,N10+N11,N00+N10)+ mutual_info(N,N01,N01+N11,N01+N00)+ mutual_info(N,N00,N00+N10,N00+N01)
            miDict[k][i] = mi
    fWords = set()
    for i in xrange(len(docCount)):
        keyf = lambda x:x[1][i]
        sortedDict = sorted(miDict.items(),key=keyf,reverse=True)
        for j in xrange(100):
            fWords.add(sortedDict[j][0])
    out = open(featureFile, 'w')
    #输出各个类的文档数目
    out.write(str(docCount)+"\n")
    #输出互信息最高的词作为特征词
    for fword in fWords:
        out.write(fword+"\n")
    print "特征词写入完毕..."
    out.close()

In [254]:
def load_feature_words(featureFile):
    '''
        从特征文件导入特征词
    '''
    f = open(featureFile)
    #各个类的文档数目
    docCounts = eval(f.readline())
    features = set()
    #读取特征词
    for line in f:
        features.add(line.strip())
    f.close()
    return docCounts,features

In [255]:
def train_bayes(featureFile, textFile, modelFile):
    '''
        训练贝叶斯模型，实际上计算每个类中特征词的出现次数
    '''
    print "使用朴素贝叶斯训练中..."
    docCounts,features = load_feature_words(featureFile)
    wordCount = collections.defaultdict(doc_dict())
    #每类文档特征词出现的次数
    tCount = [0]*len(docCounts)
    for line in open(textFile):
        lable,text = line.strip().split('\t',1)
        index = lable2id(lable[0])        
        words = text.split(' ')
        for word in words:
            if word in features:
                tCount[index] += 1
                wordCount[word][index] += 1
    outModel = open(modelFile, 'w')
    #拉普拉斯平滑
    print "训练完毕，写入模型..."
    for k,v in wordCount.items():
        scores = [(v[i]+1) * 1.0 / (tCount[i]+len(wordCount)) for i in xrange(len(v))]
        outModel.write(k+"\t"+scores+"\n")
    outModel.close()

In [256]:
def load_model(modelFile):
    '''
        从模型文件中导入计算好的贝叶斯模型
    '''
    print "加载模型中..."
    f = open(modelFile)
    scores = {}
    for line in f:
        word,counts = line.strip().rsplit('\t',1)    
        scores[word] = eval(counts)
    f.close()
    return scores

In [257]:
def predict(featureFile, modelFile, testText):
    '''
        预测文档的类标，标准输入每一行为一个文档
    '''
    docCounts,features = load_feature_words()    
    docScores = [math.log(count * 1.0 /sum(docCounts)) for count in docCounts]
    scores = load_model(modelFile)
    rCount = 0
    docCount = 0
    print "正在使用测试数据验证模型效果..."
    for line in testText:
        lable,text = line.strip().split('\t',1)
        index = lable2id(lable[0])        
        words = text.split(' ')
        preValues = list(docScores)
        for word in words:
            if word in features:                
                for i in xrange(len(preValues)):
                    preValues[i]+=math.log(scores[word][i])
        m = max(preValues)
        pIndex = preValues.index(m)
        if pIndex == index:
            rCount += 1
        #print lable,lables[pIndex],text
        docCount += 1
    print("总共测试文本量: %d , 预测正确的类别量: %d, 朴素贝叶斯分类器准确度:%f" %(rCount,docCount,rCount * 1.0 / docCount)) 

In [277]:
import sys, math, random, collections
def shuffle(inFile):
    '''
        简单的乱序操作，用于生成训练集和测试集
    '''
    textLines = [line.strip() for line in open(inFile) if len(line) > 0]
    print "正在准备训练和测试数据，请稍后..."
    random.shuffle(textLines)
    num = len(textLines)
    trainText = textLines[:3*num/5]
    testText = textLines[3*num/5:]
    print "准备训练和测试数据准备完毕，下一步..."
    return trainText, testText

In [282]:
inFile = 'data_processed2.json'
featureFile = 'feature_file.out'
modelFile = 'model_file.out'

trainText, testText = shuffle(inFile)
print trainText[0]
count_for_cates(trainText, featureFile)
train_bayes(featureFile, trainText, modelFile)
predict(featureFile, modelFile, testText)

正在准备训练和测试数据，请稍后...
准备训练和测试数据准备完毕，下一步...
体育


ValueError: need more than 1 value to unpack